In [27]:
# plot of liquity 
import scipy 
import matplotlib.pyplot as plt
import matplotlib.animation
from matplotlib.widgets import Slider
import seaborn as sns
import numpy as np 
import random
import pandas as pd 
from ipywidgets import *
from scipy.stats import norm
random.seed(1)
t =  np.linspace(91,0,92)/10
Real_vol = 0.4
Growth_rate = 2
LTV0 = 0.9
bs_vol = 0.5
y_price = 50000
Collateral = 10000000
pool_init_x = 5000000
pool_init_y = 100
pool_init_wx = 0.5
# temp = weights_generator(t, y_price, bs_vol, Growth_rate, Real_vol, ma_window=20)
# temp.describe()
#_wt_ma= np.array([1, 2, 3])
# np.append([0], (_wt_ma[:-1]/_wt_ma[1:])**_wt_ma[1:]*( (1-_wt_ma[:-1])/(1-_wt_ma[1:]))**(1-_wt_ma[1:])-1)
# np.append([np.array([1,2]), np.convolve(np.array([1,2,3,4,5]), np.ones(3)/3, mode='valid')])

In [23]:

def weights_generator(t, y_price, bs_vol, Growth_rate, Real_vol, ma_window): 
    
    """
        Purpose: generate weights based on black-shoals option pricing model and observed price. 
        
        Assumption: 
            1. Rebalnce following a delta replicating strategy of a call option on Token / Collateral. 
            2. Price percentage change (approximately equals to log(p(t+1)/p(t))) follows a linear growth treand 
               (slope = growth rate) plus a random walk Brownie motion. 
               
        Input: 
            - t: term of the loan 
            - y_price: initial price y token (e.g. 1 btc = 50000 btc) 
            - bs_vol: black shoral option volatilty 
            - Growth_rate: linear growth slope of y price 
            - Real_vol: growth Brownie motion volatility 
            - ma_window: weights moving average window.   
                         Pad zero if t < ma_window 
        Output: 
            - A dataframe with columns: ['t', 'y_price', 'weights_ytoken', 'slippage_rebalance'] 
        
    """
    if set_random_seed == True:
        np.random.seed(101)
   # loan life duation between t(i) - t(i+1)
    delta_t = t[:-1]-t[1:]
    # token price percent change following a linear growth curve plus a random walk with volatility = Real_vol 
    rw = np.random.random(size = len(t)-1)
#    delta_price = delta_t * (Growth_rate - 0.5*Real_vol**2) +  Real_vol *np.sqrt(delta_t)*np.random.normal(0, 1, len(t)-1) #norm.ppf(rw)
    delta_price = delta_t * (Growth_rate - 0.5*Real_vol**2) +  Real_vol *np.sqrt(delta_t)*norm.ppf(rw)
    
    # calcualte price with exponetial of cumulative sum of log price differences.  
    price_logdiff= np.hstack([0, delta_price]).cumsum()
    price = y_price*np.exp(price_logdiff)
    # reblance weights by black scholes option equestion. 
    w_y = norm.cdf((price_logdiff[:-1] + t[:-1]*(0.5*bs_vol**2))/bs_vol/np.sqrt(t[:-1]))
    
    # set a boundary [1e-8, 1-1e-8] to avoid 0 as denominator 
    w_y[w_y< 1e-8] = 1e-8   
    w_y[w_y> (1- 1e-8)] = 1- 1e-8
    
    weights_ytoken = np.append(w_y, w_y[-1])
    assert isinstance(ma_window, int), 'Moving average window must be integer!' 
    if ma_window == 1:
        _wt_ma = weights_ytoken.copy()
    else:
        _wt_ma = np.append(weights_ytoken[:ma_window-1], np.convolve(weights_ytoken, np.ones(ma_window)/ma_window, mode='valid'))
        #_wt_ma = np.convolve(weights_ytoken, np.ones(ma_window)/ma_window, mode='same')
    slippage = np.append([0], (_wt_ma[:-1]/_wt_ma[1:])**_wt_ma[1:]*( (1-_wt_ma[:-1])/(1-_wt_ma[1:]))**(1-_wt_ma[1:])-1)
    # formate the output file as a dataframe for downstream analysis 
    _temp= np.column_stack((t, price, _wt_ma, slippage))
    out_vec = pd.DataFrame(_temp, columns = ['t', 'y_price', 'weights_ytoken', 'slippage_rebalance'])
    return out_vec 


In [ ]:
class PoolEngine_v1: 
    """
        pool engine class
        Reflect real senario when weight changes makes arbitrage oppurtunites and pool value loss 
        
        Attribute: 
            - x: token 1 balance
            - y: token 2 balance
            - w_x: weight of x 
            - collateral value 
            - collateral with rebate 
            - fee
            - rebate 
            
        Method:
            - sp(): spot price of one y token in terms of x token
            - in-given-out(inTtype, amount): size of in given out amount 
            - out-given-in(inTtype, amount): size of our given in amount 
            - in_given_price(inTtype, p): size of in to bring spot price to p the price of out token as function of in token
                   
    """
    def __init__(self, x, y, w_x):
        assert 0<w_x<1 , "weights must be in (0,1) !"
        self.x = x
        self.y = y
        self.w_x = w_x 

    def sp(self):
        w_y = 1- self.w_x
        p = self.x*w_y/(self.y*self.w_x) 
        return p 
    
    def in_given_out(self, inTtype, amount):
        if inTtype == 'x':
            assert 0<= amount < self.y, 'out amount must be greater than 0 and less than total'
            ratio = (1-self.w_x)/self.w_x 
            Ain = self.x * ((self.y/(self.y -amount))**(ratio) -1)
        else: 
            assert 0<= amount < self.x, 'out amount must be greater than 0 and less than total'
            ratio = self.w_x/(1-self.w_x)  
            Ain = self.y * ((self.x/(self.x -amount))**(ratio) -1)
        return Ain 
    
    def out_given_in(self, inTtype, amount):
        assert amount >= 0, 'amount must be greater than 0'
        if inTtype == 'x':
            ratio = self.w_x / (1-self.w_x)
            Aout = self.y * (1- (self.x/(self.x+amount))**ratio) 
        else: 
            ratio = (1-self.w_x)/self.w_x  
            Aout = self.x * (1- (self.y/(self.y+amount))**ratio) 
        return Aout

    def in_given_price(self, inTtype, price): 
        assert price >0 , 'price must be greater than 0'
        if inTtype == 'x':
            sp_price = self.sp()
            Ain = self.x * ((price/sp_price)**(1-self.w_x) - 1) 
        else: 
            sp_price = 1/self.sp()
            Ain = self.y * ((price/sp_price)**self.w_x - 1) 
        return Ain 
    

In [ ]:
def get_episode_full(t,y_price_init, bs_vol, Growth_rate, Real_vol, Collateral, LTV0, fee_rate, rebate,
                     pool_init_x, pool_init_y, pool_init_wx, ma_window):
    
    """
    Purpuse: 
        generate one simulation based on selected parameters and random price change following the Browian motion.  
        pool attributes are rescorded at each step. 
        Rebate and fee values are back to the pool. 
    
    Input: 
        - t: term of the loan (e.g. 90 days)
        - y_price: initial price of loan token (e.g. USDC = 3e-5 btc) 
        - bs_vol: black shoral option volatilty 
        - Growth_rate: linear growth slope of y price 
        - Real_vol: growth Brownie motion volatility 
        - Collateral amount (e.g. 200 BTC) 
        - LTV0: Initial LTV (e.g. 90%)
        - fee: e.g. 0.15% of absolute weight changes 
        - rebate: e.g. 50% slippage due to rebalance weigths 
        - pool_init_x: Initial x size 
        - pool_init_y: Initial y size 
        - Pool_init_wx: Inital x weight (e.g. 50%)
        - ma_window: weights moving average window.   
                     Pad zero if t < ma_window 
    
    Output: 
        Dataframe with the following key measures. 
    
        - 1. Token price at time t 
        - 2. Reblanced weight at time t 
        - 3. Daily pool value loss (relative to collaterals)
                slippage due to rebalance weights 
                total changes due to rebalance weights and price changes 
        - 4. Collateral value 
        - 5. Collateral value with rebate 
        - 6. LTV relative to collateral
        - 7. LTV relative to collateral with rebate   

    """
    # this function replicate the CRP balance sheet.  
   


    episode = weights_generator(t, y_price_init, bs_vol, Growth_rate, Real_vol, ma_window)
    wt = episode.weights_ytoken
    oracle_price = episode.y_price
    oracle_price_ratio = np.append([1],oracle_price[1:].values/oracle_price[:-1].values)    

    borrowed_usd = Collateral*LTV0
    slippage_rebalance = episode.slippage_rebalance.values
    fee = np.abs(episode.weights_ytoken.diff(1).fillna(0).values)*fee_rate
    episode_output=[]
    with_rebate=[]
    slppage_trading=[]
    #add pool rebalancing cost and trading cost 
    #Pool starts at time 0 and run through the whole life cicle 
    rbpool = PoolEngine_v1(x = pool_init_x, y = pool_init_y, w_x = pool_init_wx)

    for i in wt.index:
            if i==0: 
                implied_p = rbpool.sp()
                btc_bal = Collateral*wt[i]
                usd_bal = Collateral*(1-wt[i])
                collateral_value = btc_bal+usd_bal
                coll_with_rebate = Collateral
                splippage_check = 0 
                rbpool.x = usd_bal 
                rbpool.y = btc_bal/oracle_price[i]
            else: 
                rbpool.w_x = 1- wt[i]
                implied_p = rbpool.sp()
                delta_x = rbpool.in_given_price('x', oracle_price[i])
                delta_y = rbpool.in_given_price('y', 1/oracle_price[i])
                btc_bal = btc_bal/oracle_price[i-1]*oracle_price[i] + delta_y*oracle_price[i-1] 
                usd_bal = usd_bal + delta_x 
                splippage_check = (delta_y*oracle_price[i]+delta_x)/collateral_value
                collateral_value = btc_bal+usd_bal
                rbpool.x = usd_bal 
                rbpool.y = btc_bal/oracle_price[i]
                coll_with_rebate = x_locked + y_locked/oracle_price[i-1]*oracle_price[i]  \
                                + coll_with_rebate*(slippage_rebalance[i] *(1-rebate)+fee[i]) 
            x_locked = coll_with_rebate*(1-wt[i])
            y_locked = coll_with_rebate* wt[i]
            with_rebate.append((x_locked, y_locked, coll_with_rebate, btc_bal, usd_bal, collateral_value, splippage_check))  

    _with_rebate = pd.DataFrame(np.array(with_rebate), columns= ['x_locked', 'y_locked', 'coll_with_rebate',\
                                            'btc_bal', 'usd_bal', 'collateral_value', 'splippage_check'])
    #collateral value in usd 
    episode_full = pd.concat([episode, _with_rebate], axis=1)
    # collateral value with re 
    episode_full['ltv_rebalance'] = borrowed_usd/(episode_full['collateral_value'])
    episode_full['ltv_with_rebate'] = borrowed_usd/episode_full['coll_with_rebate']    
    episode_full['wt_chg'] = episode_full['weights_ytoken'].diff(1).fillna(0)
    episode_full['pvtc_rebate'] = _with_rebate['coll_with_rebate'].values/episode_full['y_price'].values/(Collateral/y_price_init)

    return episode_full


In [42]:
def episode_plot(var_list, set_random_seed = True, row=2, col=3, Real_vol = 0.5, Growth_rate = 1, LTV0 = 0.8, bs_vol = 0.5,
    y_price_init = 50000, Collateral = 1000000, pool_init_x = 5000000, pool_init_y = 100, pool_init_wx = 0.5,
    fee_rate = 0.0015,  rebate=0.5, ma_window =7):
    %matplotlib notebook
    import matplotlib.pyplot as plt 
    fig, axs = plt.subplots(row,col,figsize=(8,6))
    axs[-1, -1].axis('off')
    # Create axes for sliders
    ax_growth = fig.add_axes([0.73, 0.3, 0.2, 0.02])
    ax_growth.spines['top'].set_visible(True)
    ax_growth.spines['right'].set_visible(True)

    ax_real_vol = fig.add_axes([0.73, 0.26, 0.2, 0.02])
    ax_real_vol.spines['top'].set_visible(True)
    ax_real_vol.spines['right'].set_visible(True)

    ax_ltv = fig.add_axes([0.73, 0.22, 0.2, 0.02])
    ax_ltv.spines['top'].set_visible(True)

    ax_bs_vol = fig.add_axes([0.73, 0.18, 0.2, 0.02])
    ax_bs_vol.spines['top'].set_visible(True)
    ax_bs_vol.spines['right'].set_visible(True)

    ax_rebate = fig.add_axes([0.73, 0.14, 0.2, 0.02])
    ax_rebate.spines['top'].set_visible(True)
    ax_rebate.spines['right'].set_visible(True)

    ax_ma = fig.add_axes([0.73, 0.10, 0.2, 0.02])
    ax_ma.spines['top'].set_visible(True)
    ax_ma.spines['right'].set_visible(True)
    
    ax_fee = fig.add_axes([0.73, 0.34, 0.2, 0.02])
    ax_fee.spines['top'].set_visible(True)
    ax_fee.spines['right'].set_visible(True)

    # Create sliders
    s_real_vol= Slider(ax=ax_real_vol, label='Real Vol', valmin=0.1, valmax=1.0, valinit=0.5, valfmt=' %1.2f ', facecolor='#cc7000')
    s_growth = Slider(ax=ax_growth, label='Growth', valmin=-2, valmax=2, valinit=1, valfmt=' %1.2f', facecolor='#cc7000')
    s_bs_vol= Slider(ax=ax_bs_vol, label='BS Vol', valmin=0.1, valmax=1.0, valinit=0.5, valfmt=' %1.2f ', facecolor='#cc7000')
    s_ltv = Slider(ax=ax_ltv, label='LTV0', valmin=0.7, valmax=0.95, valinit=0.9, valfmt=' %1.2f', facecolor='#cc7000')
    s_rebate = Slider(ax=ax_rebate, label='Rebate', valmin=0.0, valmax=1, valinit=0.5, valfmt=' %1.2f', facecolor='#cc7000')
    s_ma =     Slider(ax=ax_ma, label='MA', valmin=1, valmax=30, valinit=7, valfmt=' %0.0f ', facecolor='#cc7000')
    s_fee = Slider(ax=ax_fee, label='Fee', valmin=0.0, valmax=0.01, valinit=0.0015, valfmt=' %1.4f', facecolor='#cc7000')


    # Plot default data
    t =  np.linspace(91,0,92)/365
    _data = get_episode_full(t,y_price_init, bs_vol, Growth_rate, Real_vol, Collateral, LTV0, fee_rate, rebate,
                     pool_init_x, pool_init_y, pool_init_wx, ma_window)

    x = _data.index.values
    y1 = _data[var_list[0]]/y_price_init
    y2a = _data[var_list[1]]
    y2b = 1-_data[var_list[1]]    
    y3 = _data[var_list[2]]
    y4 = _data[var_list[3]]
    y5 = _data[var_list[4]]
    y6 = _data[var_list[5]]

    f_d1, = axs[0,0].plot(x, y1, linewidth=2.5)
    f_d2a, = axs[0,1].plot(x, y2a, linewidth=2.5)
    f_d2b, = axs[0,1].plot(x, y2b, linewidth=2.5)
    f_d3, = axs[0,2].plot(x, y3, linewidth=2.5)
    f_d4, = axs[1,0].plot(x, y4, linewidth=2.5)
    f_d5, = axs[1,1].plot(y6, y5, marker= 'o', linestyle = 'None', linewidth=1)


    axs[0,0].set_ylim(0.5,2.5)
    axs[0,0].axhline(y = 1, color = 'r', linestyle = '--')
    axs[0,1].set_ylim(-0.02,1.02)
    axs[0,1].axhline(y = 0.5, color = 'r', linestyle = '--')
    axs[0,2].set_ylim(0.5,2)
    axs[0,2].axhline(y = 1, color = 'r', linestyle = '--')
    axs[1,0].set_ylim(0.5,2)
    axs[1,0].axhline(y = 1, color = 'r', linestyle = '--')
    axs[1,1].set_ylim(-0.2,0.05)
    axs[1,1].set_xlim(-0.5,0.5)
    axs[1,1].axhline(y = 0, color = 'r', linestyle = '--')
   
    
    axs[0,0].set_ylabel('BTC relative change ')
    axs[0,1].set_ylabel('Weights (USDC: Orange, BTC: Blue)')
    axs[0,2].set_ylabel('LTV-with rebate')
    axs[1,0].set_ylabel('PVTC with rebate') 
    axs[1,1].set_ylabel('Impermanent Loss vs weight chg') 

    for i in range(2):
        for j in range(3):
            axs[i,j].set_xlabel('Day')
            axs[i,j].set_title('({})'.format(3*i+j+1))
    axs[1,1].set_xlabel('weight changes')
    # Update values
    def update(val):
        _real_vol = s_real_vol.val
        _growth = s_growth.val
        _bs_vol = s_bs_vol.val
        _ltv = s_ltv.val
        _rebate = s_rebate.val
        _ma = int(s_ma.val)
        _fee = s_fee.val
        _data = get_episode_full(t,y_price_init, _bs_vol, _growth, _real_vol, Collateral, _ltv, _fee, _rebate,
                         pool_init_x, pool_init_y, pool_init_wx, _ma)
        x = _data.index.values
        _y1 = _data[var_list[0]]/y_price_init
        _y2a = _data[var_list[1]]
        _y2b = 1-_data[var_list[1]]    
        _y3 = _data[var_list[2]]
        _y4 = _data[var_list[3]]
        _y5 = _data[var_list[4]]
        _y6 = _data[var_list[5]]

        f_d1.set_data(x,_y1)
        f_d2a.set_data(x,_y2a)
        f_d2b.set_data(x,_y2b)
        f_d3.set_data(x,_y3)
        f_d4.set_data(x,_y4)
        f_d5.set_data(_y6,_y5)
        fig.canvas.draw_idle()

    s_real_vol.on_changed(update)
    s_growth.on_changed(update)
    s_ltv.on_changed(update)
    s_bs_vol.on_changed(update)
    s_rebate.on_changed(update)
    s_ma.on_changed(update)
    s_fee.on_changed(update)
    fig.tight_layout()
    plt.show()
